# Classification/Testing
- Run this script to classify images in the testing directory specified as `test_dir` using the master catalog generated from training
- Outputs will be generated in the `outputs/classification_results` directory and will include:
    - `data` directory with `.json` and `.txt` files that have breakdowns for each image classified
    - `.csv` file containing overall pixel area and layer thickness breakdowns for all images 
    - `images` directory with visualizations of each test image and its classified layer results as a `.png` file
      - `images` enabled by default but can be optionally disabled if unneeded to speed up processing of large image datasets

Final data outputs are `.csv` and `.json` for easy post-processing and statistical analysis using data analytics tools such as Pandas, R, Excel, or MATLAB.

Notes:
  - `test_dir` is the directory where images to be classified should be located
  - `master_cat_file` in `base_args` should be the file path for the master catalog 

In [2]:
import os
import glob
from utils.clustering_core import testing
from utils.data_management import consolidate_json_to_csv


    
# Directory containing test images
test_dir = 'image_data/testing/'

# Common parameters for all images
base_args = {
    'master_cat_file': "outputs/training/master_catalog/mos2_on_pdms_master_catalog.npz",
    'comp_rate': None,
    'cluster_count': None,  # Will use all available clusters
    'show_plot': False,  # Set to True to display the classification results in notebook output
    # 'save_images': False  # Set to False will skip saving the classification visualizations as images
}

# Get all image files (common formats)
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tif', '*.tiff']
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(test_dir, ext)))
    image_files.extend(glob.glob(os.path.join(test_dir, ext.upper())))

# Remove duplicates and sort
image_files = sorted(list(set(image_files)))

if len(image_files) == 0:
    print(f"No image files found in {test_dir}")
else:
    print(f"Found {len(image_files)} images to process")
    
    # Process each image
    for i, img_file in enumerate(image_files, 1):
        print(f"\n{'='*60}")
        print(f"Processing image {i}/{len(image_files)}: {os.path.basename(img_file)}")
        print(f"{'='*60}")
        
        try:
            # Create args for this specific image
            args = base_args.copy()
            args['img_file'] = img_file
            
            # Process the image
            results = testing(**args)
            
            print(f"✓ Successfully processed {os.path.basename(img_file)}")
            
        except Exception as e:
            print(f"✗ Error processing {os.path.basename(img_file)}: {str(e)}")
            continue
    
    print(f"\n{'='*60}")
    print(f"Batch processing complete!")
    print(f"Results saved to ./outputs/classification_results/")

    # Create csv from json files
    input_directory = "./outputs/classification_results/data"
    output_directory = "./outputs/classification_results"
    consolidate_json_to_csv(input_directory, output_directory)

Found 1 images to process

Processing image 1/1: Fig_4f.jpg
Processing image: Fig_4f
Original image shape: (3488, 4632, 3)
Applying bilateral filtering...
Detecting background regions automatically...
Fitting and subtracting background...
Applying second round of bilateral filtering...
Processing 16156416 valid pixels out of 16156416 total pixels
Available layers in master catalog: [0, 1, 2, 3]
Loaded data for 0 layers (cluster index 0)
Loaded data for 1 layers (cluster index 1)
Loaded data for 2 layers (cluster index 2)
Loaded data for 3 layers (cluster index 3)
Calculating probabilities for 16156416 valid pixels using GPU...
GPU probability calculation completed in 0.25 seconds
Time elapsed: 3.48 seconds
Saved visualization to ./outputs/classification_results/images/Fig_4f_analysis.png
Saved data to ./outputs/classification_results/data/Fig_4f_data.json and ./outputs/classification_results/data/Fig_4f_summary.txt
Results saved to ./outputs/classification_results/
✓ Successfully proce